<a href="https://colab.research.google.com/github/mion158/Language-and-Chatbot/blob/main/Viet_Eng.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

preprocess the text file downloaded from: http://www.manythings.org/anki/

In [1]:
import numpy as np
import re

# Importing our translations
# for example: "spa.txt" or "spa-eng/spa.txt"
data_path = "vie.txt"

# Defining lines as a list of each line
with open(data_path, 'r', encoding='utf-8') as f:
  lines = f.read().split('\n')

lines

['Run!\tChạy!\tCC-BY 2.0 (France) Attribution: tatoeba.org #906328 (papabear) & #1541945 (Aomori)',
 'Help!\tGiúp tôi với!\tCC-BY 2.0 (France) Attribution: tatoeba.org #435084 (lukaszpp) & #5489308 (phuc)',
 'Go on.\tTiếp tục đi.\tCC-BY 2.0 (France) Attribution: tatoeba.org #2230774 (CK) & #9203952 (dotheduyet1999)',
 'Hello!\tChào bạn.\tCC-BY 2.0 (France) Attribution: tatoeba.org #373330 (CK) & #510482 (tucm)',
 'Hurry!\tNhanh lên nào!\tCC-BY 2.0 (France) Attribution: tatoeba.org #1182334 (Hideousss) & #5489313 (phuc)',
 'I try.\tTôi sẽ thử.\tCC-BY 2.0 (France) Attribution: tatoeba.org #20776 (CK) & #3606826 (khoaharp)',
 'I try.\tTôi sẽ thử xem sao.\tCC-BY 2.0 (France) Attribution: tatoeba.org #20776 (CK) & #5147615 (truongap)',
 'Eat it.\tĂn đi.\tCC-BY 2.0 (France) Attribution: tatoeba.org #4768107 (garborg) & #8957153 (dotheduyet1999)',
 'Eat it.\tĂn nó đi.\tCC-BY 2.0 (France) Attribution: tatoeba.org #4768107 (garborg) & #8957154 (dotheduyet1999)',
 'Got it!\tHiểu rồi!\tCC-BY 2.0 

In [5]:
# Building empty lists to hold sentences
input_docs = []
target_docs = []
# Building empty vocabulary sets
input_tokens = set()
target_tokens = set()

# Adjust the number of lines so that preprocessing doesn't take too long for you
for line in lines[:501]:
  # Input and target sentences are separated by tabs
  input_doc, target_doc = line.split('\t')[:2]
  # Appending each input sentence to input_docs
  input_docs.append(input_doc)

  target_doc = " ".join(re.findall(r"[\w']+|[^\s\w]", target_doc))
  # Redefine target_doc below
  # and append it to target_docs:
  target_doc = '<START> ' + target_doc + ' <END>'
  target_docs.append(target_doc)

  # Now we split up each sentence into words
  # and add each unique word to our vocabulary set
  for token in re.findall(r"[\w']+|[^\s\w]", input_doc):
    # print(token)
    # Add your code here:
    if token not in input_tokens:
      input_tokens.add(token)
  for token in target_doc.split():
    # print(token)
    # And here:
    if token not in target_tokens:
      target_tokens.add(token)

input_tokens = sorted(list(input_tokens))
target_tokens = sorted(list(target_tokens))

print(input_tokens)
print(target_tokens)

['!', '$', ',', '.', '100', '2', '30', ':', '?', 'After', 'Anybody', 'Are', 'Ask', 'At', 'Bach', 'Be', 'Birds', 'Boston', 'CD', 'Call', 'Can', 'Cheer', 'Christmas', 'Come', 'Congratulations', 'Did', 'Do', 'Does', "Don't", 'Drive', 'Eat', 'Even', 'Everyone', 'Find', 'Ford', 'French', 'Get', 'Give', 'Go', 'Good', 'Goodbye', 'Got', 'Hang', 'He', 'Hello', 'Help', 'Here', 'How', 'Hurry', 'I', "I'd", "I'll", "I'm", 'Is', "Isn't", 'It', "It'll", "It's", 'Just', 'Keep', 'Leave', 'Let', "Let's", 'Life', "Life's", 'Look', 'Mary', 'May', 'Men', 'Merry', 'Monday', 'My', 'Natto', 'No', 'Nobody', 'Now', 'OK', 'Oh', 'Perfect', 'Play', 'Run', 'She', "She's", 'Should', 'Sit', 'Spit', 'Stay', 'Stop', 'Suit', 'Sweep', 'TV', 'Take', 'Talk', 'Tell', 'That', "That's", 'Their', "There's", 'They', "They'll", "They're", 'This', 'Those', 'Time', 'Tom', "Tom's", 'Too', 'Trust', 'UFO', 'Unbelievable', 'Vote', "Wasn't", 'Watch', 'We', "We'll", "We're", "What's", 'Where', "Where's", 'Who', "Who's", 'Why', "Won't", 

In [10]:
# Create num_encoder_tokens and num_decoder_tokens:
num_encoder_tokens = len(input_tokens)
num_decoder_tokens = len(target_tokens)

max_encoder_seq_length = max([len(re.findall(r"[\w']+|[^\s\w]", input_doc)) for input_doc in input_docs])
max_decoder_seq_length = max([len(re.findall(r"[\w']+|[^\s\w]", target_doc)) for target_doc in target_docs])

input_features_dict = dict(
    [(token, i) for i, token in enumerate(input_tokens)])
target_features_dict = dict(
    [(token, i) for i, token in enumerate(target_tokens)])

reverse_input_features_dict = dict(
    (i, token) for token, i in input_features_dict.items())
reverse_target_features_dict = dict(
    (i, token) for token, i in target_features_dict.items())

encoder_input_data = np.zeros(
    (len(input_docs), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_docs), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_docs), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

for line, (input_doc, target_doc) in enumerate(zip(input_docs, target_docs)):

  for timestep, token in enumerate(re.findall(r"[\w']+|[^\s\w]", input_doc)):
    # Assign 1. for the current line, timestep, & word in encoder_input_data:
    encoder_input_data[line, timestep, input_features_dict[token]] = 1.

  for timestep, token in enumerate(target_doc.split()):

    decoder_input_data[line, timestep, target_features_dict[token]] = 1.
    if timestep > 0:

      decoder_target_data[line, timestep - 1, target_features_dict[token]] = 1.

print(list(input_features_dict.keys())[:50])
print(len(input_tokens))

['!', '$', ',', '.', '100', '2', '30', ':', '?', 'After', 'Anybody', 'Are', 'Ask', 'At', 'Bach', 'Be', 'Birds', 'Boston', 'CD', 'Call', 'Can', 'Cheer', 'Christmas', 'Come', 'Congratulations', 'Did', 'Do', 'Does', "Don't", 'Drive', 'Eat', 'Even', 'Everyone', 'Find', 'Ford', 'French', 'Get', 'Give', 'Go', 'Good', 'Goodbye', 'Got', 'Hang', 'He', 'Hello', 'Help', 'Here', 'How', 'Hurry', 'I']
455


now train the model

In [11]:
from tensorflow import keras
# Add Dense to the imported layers
from keras.layers import Input, LSTM, Dense
from keras.models import Model

import os

# Choose a dimensionality
latent_dim = 256

# Choose a batch size
# and a larger number of epochs:
batch_size = 50
epochs = 100


In [12]:
# Encoder training setup
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_hidden, state_cell = encoder_lstm(encoder_inputs)
encoder_states = [state_hidden, state_cell]

# Decoder training setup:
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, decoder_state_hidden, decoder_state_cell = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [13]:
# Building the training model:
training_model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

print("Model summary:\n")
training_model.summary()
print("\n")

# Compile the model:
training_model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

print("Training the model:\n")
# Train the model:
training_model.fit([encoder_input_data, decoder_input_data], decoder_target_data, batch_size = batch_size, epochs = epochs, validation_split = 0.2)

training_model.save('training_model.h5')

Model summary:

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, 455)]  0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, None, 554)]  0           []                               
                                                                                                  
 lstm (LSTM)                    [(None, 256),        729088      ['input_1[0][0]']                
                                 (None, 256),                                                     
                                 (None, 256)]                                                     
                                                                              

Now test the model

In [14]:
from keras.models import load_model

#same notebook but save and load model just in case
training_model = load_model('training_model.h5')
#for loading model
encoder_inputs = training_model.input[0]
encoder_outputs, state_h_enc, state_c_enc = training_model.layers[2].output
encoder_states = [state_h_enc, state_c_enc]

encoder_model = Model(encoder_inputs, encoder_states)

In [15]:
decoder_state_input_hidden = Input(shape=(latent_dim,))
decoder_state_input_cell = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_hidden, decoder_state_input_cell]
decoder_outputs, state_hidden, state_cell = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_hidden, state_cell]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

In [16]:
def decode_sequence(test_input):
  # Encode the input as state vectors.
  states_value = encoder_model.predict(test_input)

  # Generate empty target sequence of length 1.
  target_seq = np.zeros((1, 1, num_decoder_tokens))
  # Populate the first token of target sequence with the start token.
  target_seq[0, 0, target_features_dict['<START>']] = 1.

  # Sampling loop for a batch of sequences (to simplify, here we assume a batch of size 1).
  decoded_sentence = ''

  stop_condition = False
  while not stop_condition:
    # Run the decoder model to get possible 
    # output tokens (with probabilities) & states
    output_tokens, hidden_state, cell_state = decoder_model.predict(
      [target_seq] + states_value)

    # Choose token with highest probability
    sampled_token_index = np.argmax(output_tokens[0, -1, :])
    sampled_token = reverse_target_features_dict[sampled_token_index]
    decoded_sentence += " " + sampled_token

    # Exit condition: either hit max length
    # or find stop token.
    if (sampled_token == '<END>' or len(decoded_sentence) > max_decoder_seq_length):
      stop_condition = True

    # Update the target sequence (of length 1).
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, sampled_token_index] = 1.

    # Update states
    states_value = [hidden_state, cell_state]

  return decoded_sentence


for seq_index in range(100):
  test_input = encoder_input_data[seq_index: seq_index + 1]
  decoded_sentence = decode_sequence(test_input)
  print('-')
  print('Input sentence:', input_docs[seq_index])
  print('Decoded sentence:', decoded_sentence)

-
Input sentence: Run!
Decoded sentence:  Chạy . <END>
-
Input sentence: Help!
Decoded sentence:  Đi tôi tôi . <END>
-
Input sentence: Go on.
Decoded sentence:  Cứ tục đi . . <END>
-
Input sentence: Hello!
Decoded sentence:  Chào . . <END>
-
Input sentence: Hurry!
Decoded sentence:  Hoàn hảo ! <END>
-
Input sentence: I try.
Decoded sentence:  Tôi sẽ thử . <END>
-
Input sentence: I try.
Decoded sentence:  Tôi sẽ thử . <END>
-
Input sentence: Eat it.
Decoded sentence:  Ăn đi . . <END>
-
Input sentence: Eat it.
Decoded sentence:  Ăn đi . . <END>
-
Input sentence: Got it!
Decoded sentence:  Hiểu đi <END>
-
Input sentence: Goodbye!
Decoded sentence:  Chúc mừng <END>
-
Input sentence: Help me!
Decoded sentence:  Hãy tôi tôi . <END>
-
Input sentence: I agree.
Decoded sentence:  Tôi sẽ thử . . <END>
-
Input sentence: Perfect!
Decoded sentence:  Chúc mừng <END>
-
Input sentence: We know.
Decoded sentence:  Chúng tôi tôi . <END>
-
Input sentence: We know.
Decoded sentence:  Chúng tôi tôi . <END>